In [3]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression


Reading the train data , weather data and key data 

In [4]:
lm = LinearRegression()
train_df = pd.read_csv('train.csv')
weather_df = pd.read_csv('weather.csv')
key_df = pd.read_csv('key.csv')
print(train_df.head())

         date  store_nbr  item_nbr  units
0  2012-01-01          1         1      0
1  2012-01-01          1         2      0
2  2012-01-01          1         3      0
3  2012-01-01          1         4      0
4  2012-01-01          1         5      0


Merge key data and weather data on station number and also merge this data with train data on station number and date

Taking only first 10000 rows due to memory error



In [8]:

#comb_1_df = pd.merge(key_df, weather_df, on='station_nbr')

comb_1_df = pd.merge(key_df.head(10000), weather_df.head(10000), on='station_nbr')
# print(comb_1_df.head())

comb_2_df = pd.merge(train_df.head(10000), comb_1_df, on=['store_nbr', 'date'])
#print(comb_2_df.info())
train_final_y_df = comb_2_df['units']
# print(len(train_final_y_df))
train_final_x_df = comb_2_df.drop(['station_nbr', 'units'], axis=1)
print(train_final_x_df.head())
# print(train_final_x_df.info())
#lm.fit(train_final_x_df, train_final_y_df)
# print(lm.intercept_)

         date  store_nbr  item_nbr tmax tmin tavg depart dewpoint wetbulb  \
0  2012-01-01          1         1   52   31   42      M       36      40   
1  2012-01-01          1         2   52   31   42      M       36      40   
2  2012-01-01          1         3   52   31   42      M       36      40   
3  2012-01-01          1         4   52   31   42      M       36      40   
4  2012-01-01          1         5   52   31   42      M       36      40   

  heat   ...    sunrise sunset     codesum snowfall preciptotal stnpressure  \
0   23   ...          -      -  RA FZFG BR        M        0.05       29.78   
1   23   ...          -      -  RA FZFG BR        M        0.05       29.78   
2   23   ...          -      -  RA FZFG BR        M        0.05       29.78   
3   23   ...          -      -  RA FZFG BR        M        0.05       29.78   
4   23   ...          -      -  RA FZFG BR        M        0.05       29.78   

  sealevel resultspeed resultdir avgspeed  
0    29.92        

Lets concentrate on the missing values in the input data especially for weather data containing tmin,tmax and tavg

+ Assigning input data frame 'train_final_x_df' into another data frame as back up before modifying
+ fiinding any missing data in tmax,tmin and tavg
+ For the first 10000 values there are no missing values in tmin and tmax
+ however 223 rows has missing values 'M' in tavg
+ interpolate the missing values in tavg as the average of tmin and tmax


In [12]:

few_comb2_df=train_final_x_df

#Data finishing
#fiinding any missing data in tmax,tmin and tavg by finding rows that contains value M in their columns
#prinnting is optional just to have a look number of missing data
#for zero output indicates no missing data

#tmax
few_comb2_df_tmax=few_comb2_df[few_comb2_df['tmax']=='M']
#print(few_comb2_df_tmax) #output as zero rows of missing data - M


#tmin
few_comb2_df_tmin=few_comb2_df[few_comb2_df['tmin']=='M']
#print(few_comb2_df_tmin) #output as zero rows of missing data - M


#tavg
few_comb2_df_tavg=few_comb2_df[few_comb2_df['tavg']=='M']
#print(few_comb2_df_tavg.head()) #output as 223 rows of misssing data - M


#replace mssing data of tavg with average of tmin and tmax 
few_comb2_df_tavg=few_comb2_df[few_comb2_df['tavg']=='M']
few_comb2_df_tavg['tavg']=((few_comb2_df_tavg['tmax'].astype(int)+few_comb2_df_tavg['tmin'].astype(int))/2).astype(int)
#print(few_comb2_df_tavg['tavg'])
few_comb2_df_tavg.head()


Empty DataFrame
Columns: [date, store_nbr, item_nbr, tmax, tmin, tavg, depart, dewpoint, wetbulb, heat, cool, sunrise, sunset, codesum, snowfall, preciptotal, stnpressure, sealevel, resultspeed, resultdir, avgspeed]
Index: []

[0 rows x 21 columns]


C:\Users\Anand\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,date,store_nbr,item_nbr,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,...,sunrise,sunset,codesum,snowfall,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed
4107,2012-01-01,39,1,66,45,55,M,34,46,M,...,-,-,RA BR,M,0.00,30.05,M,11.0,36,10.9
4108,2012-01-01,39,2,66,45,55,M,34,46,M,...,-,-,RA BR,M,0.00,30.05,M,11.0,36,10.9
4109,2012-01-01,39,3,66,45,55,M,34,46,M,...,-,-,RA BR,M,0.00,30.05,M,11.0,36,10.9
4110,2012-01-01,39,4,66,45,55,M,34,46,M,...,-,-,RA BR,M,0.00,30.05,M,11.0,36,10.9
4111,2012-01-01,39,5,66,45,55,M,34,46,M,...,-,-,RA BR,M,0.00,30.05,M,11.0,36,10.9


Lets look for missing data -M in the dewpoint in the same way as above

In [15]:
df_dew= few_comb2_df_tavg[few_comb2_df_tavg['dewpoint']=='M']
print(df_dew)

Empty DataFrame
Columns: [date, store_nbr, item_nbr, tmax, tmin, tavg, depart, dewpoint, wetbulb, heat, cool, sunrise, sunset, codesum, snowfall, preciptotal, stnpressure, sealevel, resultspeed, resultdir, avgspeed]
Index: []

[0 rows x 21 columns]


In [ ]:
No missing data in first 10000 rows